# This is the demo of the DR-MASE for the ABCD data application

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn import linear_model
from sklearn import model_selection
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier, MLPRegressor
import sklearn.metrics as metrics
from sklearn.ensemble import StackingRegressor, StackingClassifier
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
import multiprocess
from scipy import linalg, special
import pandas as pd
import statsmodels.formula.api as smf
import statistics as stat
import seaborn as sns
import time
from sklearn.ensemble import VotingClassifier, VotingRegressor
from sklearn.naive_bayes import GaussianNB
import os 

## for both PS and ICE estimate, you will need the covariates sets (cov set for each time point for PS and ICE), and the treatment and outcome names
### We provided the code for two and three time points measure

In [ ]:
# PS function
def ps_train_ensemble(data,Train,Test,features,outcomes,
                      include_lg=True, include_el=True, include_xgb=True, include_mlp=False,
                      include_l1=False, include_svm=False, include_nb=False,include_rf=False):
    import warnings
    warnings.filterwarnings('ignore')
    # Train,Test = train_test_split(data,test_size=0.4,random_state=42)
    np.random.seed(233)
    models = {}
    # base learner 1
    if include_lg:
        lg = linear_model.LogisticRegression()
        lg.fit(data[features],data[outcomes])
        models['logit'] = lg

    # base learner 2
    if include_el:
        el_class = linear_model.SGDClassifier(penalty='elasticnet',loss='log')
        param_grid_el = {'alpha': [0.0001, 0.001, 0.01, 0.1]}
        gs_el = GridSearchCV(estimator=el_class,param_grid=param_grid_el,cv=5,scoring='neg_log_loss')
        gs_el.fit(Train[features],Train[outcomes])
        el_class.set_params(**gs_el.best_params_)
        el_class.fit(Train[features],Train[outcomes])
        models['enet'] = el_class

    # base learner 3
    if include_xgb:
        param_grid_G = {
        'learning_rate': [0.01, 0.1, 1],
         'max_depth': [55,65,80],
        # 'n_estimators': [100,200,300],
        # 'alpha':[0.05,0.1,0.18],
        # 'gamma': [1,1.5,1.8]
        }
        xgb_cls = xgb.XGBClassifier(objective='binary:logistic')
        gs_xgb = GridSearchCV(
            estimator=xgb_cls,
            param_grid=param_grid_G,
            cv=5,
            scoring='neg_log_loss'
        )
        gs_xgb.fit(Train[features], Train[outcomes])
        xgb_cls.set_params(**gs_xgb.best_params_)
        xgb_cls.fit(Train[features], Train[outcomes])
        models['xgb'] = xgb_cls

    # base learner 4
    if include_mlp:
        mlp_cls = MLPClassifier()
        param_grid_mlpc = {
            'hidden_layer_sizes': [(50,), (100,), (50,50), (100,100)],
        }
        gs_mlp = GridSearchCV(estimator=mlp_cls,param_grid=param_grid_mlpc,cv=5,scoring="neg_log_loss")
        gs_mlp.fit(Train[features],Train[outcomes])
        mlp_cls.set_params(**gs_mlp.best_params_)
        mlp_cls.fit(Train[features],Train[outcomes])
        models['mlp'] = mlp_cls

    if include_l1:
        l1_cls = linear_model.LogisticRegression(penalty='l1',solver='saga')
        param_grid_l1 = {
        'C':[0.1,1,5],
        #'max_features': [None, 'sqrt', 'log2']
        }
        gs_l1 = GridSearchCV(estimator=l1_cls,param_grid=param_grid_l1,cv=5,scoring="neg_log_loss")
        gs_l1.fit(Train[features],Train[outcomes])
        l1_cls.set_params(**gs_l1.best_params_)
        l1_cls.fit(Train[features],Train[outcomes])
        models['gbr'] = l1_cls

    if include_svm:
        svm_c = SVC(kernel='linear',probability=True)
        gs_svm = GridSearchCV(estimator=svm_c,param_grid={'C': [0.1,1,10,100]},cv=5,scoring='neg_log_loss')
        gs_svm.fit(Train[features],Train[outcomes])
        svm_c.set_params(**gs_svm.best_params_)
        svm_c.fit(Train[features],Train[outcomes])
        models['svm'] = svm_c
        
    if include_nb:
        nb_c = GaussianNB()
        param_grid_nb = {
            'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]
            }
        gs_nb = GridSearchCV(nb_c, param_grid=param_grid_nb, cv=5)
        gs_nb.fit(Train[features],Train[outcomes])
        nb_c.set_params(**gs_nb.best_params_)
        nb_c.fit(Train[features],Train[outcomes])
        models['nb'] = nb_c

    if include_rf:
        rf_c = RandomForestClassifier(random_state = 42)
        param_grid_rf = {
            'n_estimators':[100,200],
            'max_depth':[40,50],
            'min_samples_split':[2,5],
            }
        gs_rf = GridSearchCV(rf_c, param_grid=param_grid_rf, cv=5, scoring='neg_log_loss')
        gs_rf.fit(Train[features],Train[outcomes])
        rf_c.set_params(**gs_rf.best_params_)
        rf_c.fit(Train[features],Train[outcomes])
        models['rf'] = rf_c

    # meta learner
    Q1fit = []
    if include_lg:
        Q1fit.append(lg.predict_proba(Test[features])[:,1].ravel())
    if include_el:
        Q1fit.append(el_class.predict_proba(Test[features])[:,1].ravel())
    if include_xgb:
        Q1fit.append(xgb_cls.predict_proba(Test[features])[:,1].ravel())
    if include_mlp:
        Q1fit.append(mlp_cls.predict_proba(Test[features])[:,1].ravel())
    if include_l1:
        Q1fit.append(l1_cls.predict_proba(Test[features])[:,1].ravel())
    if include_svm:
        Q1fit.append(svm_c.predict_proba(Test[features])[:,1].ravel())
    if include_nb:
        Q1fit.append(nb_c.predict_proba(Test[features])[:,1].ravel())
    if include_rf:
        Q1fit.append(rf_c.predict_proba(Test[features])[:,1].ravel())

    cls_stack = linear_model.LogisticRegression()
    cls_stack.fit(np.transpose(Q1fit),Test[outcomes])

    return models, cls_stack

def ps_function(data,features,outcomes, lg=True, el=True, xgb=True, mlp=False,
                l1=False, svm=False, nb=False,rf=False):
    import warnings
    warnings.filterwarnings('ignore')
    Train,Test = train_test_split(data,test_size=0.4,random_state=42)
    model_tr,stack_tr = ps_train_ensemble(data,Train,Test,features,outcomes, include_lg=lg, include_el=el, include_xgb=xgb, include_mlp=mlp,
                                          include_l1=l1, include_svm=svm, include_nb=nb,include_rf=rf)
    model_ts,stack_ts = ps_train_ensemble(data,Test,Train,features,outcomes, include_lg=lg, include_el=el, include_xgb=xgb, include_mlp=mlp,
                                          include_l1=l1, include_svm=svm, include_nb=nb,include_rf=rf)
    
    # def ps_predict(preddat,features,model_tr,model_ts,stack_tr,stack_ts):
    y_pred_tr = {}
    for name, model in model_tr.items():
        y_pred_tr[name] = model.predict_proba(Test[features])[:,1].ravel()
    Q_base_tr = np.transpose(np.array(list(y_pred_tr.values())))
    eta_tr = stack_tr.predict_proba(Q_base_tr)[:,1]

    y_pred_ts = {}
    for name, model in model_ts.items():
        y_pred_ts[name] = model.predict_proba(Train[features])[:,1].ravel()
    Q_base_ts = np.transpose(np.array(list(y_pred_ts.values())))
    eta_ts = stack_ts.predict_proba(Q_base_ts)[:,1]

    eta_tr_series = pd.Series(eta_tr, index=Test.index)
    eta_ts_series = pd.Series(eta_ts, index=Train.index)
    eta = pd.concat([eta_tr_series, eta_ts_series]).sort_index()

    # return eta_tr,eta_ts
    return model_tr,model_ts,stack_tr,stack_ts,eta_tr,eta_ts,eta



In [ ]:
# Gcomputation function

def gcompu_train_ensemble(Train,Test,features,outcomes,
                          include_lm = True, include_rg = False, 
                          include_xgb = True, include_mlp = False,
                          include_gbr = False):
    import warnings
    warnings.filterwarnings('ignore')
    # Train,Test = train_test_split(data,test_size=0.4,random_state=42)
    np.random.seed(233)
    models = {}
    # base learner 1
    if include_lm:
        lm_1 = linear_model.LinearRegression()
        lm_1.fit(Train[features],Train[outcomes])
        models['lm'] = lm_1

    # base learner 2
    if include_rg:
        rg_lm = linear_model.Ridge()
        param_grid_rg = {'alpha': [0.1, 1, 10]}
        gs_rg = GridSearchCV(estimator=rg_lm,param_grid=param_grid_rg,cv=5,scoring="neg_mean_absolute_error")
        gs_rg.fit(Train[features],Train[outcomes])
        rg_lm.set_params(**gs_rg.best_params_)
        rg_lm.fit(Train[features],Train[outcomes])
        models['rg'] = rg_lm

    # base learner 3
    if include_xgb:
        param_grid_G = {
        'learning_rate': [0.01, 0.1, 1],
        'max_depth': [15,20,25,35],
        # 'n_estimators': [100, 150,200],
        #'alpha':[0.05,0.1,0.18],
        #'gamma': [1,1.5,1.8]
        }
        xgb_lm1_tr = xgb.XGBRegressor()
        gs_gcomp1_tr = GridSearchCV(
            estimator=xgb_lm1_tr,
            param_grid=param_grid_G,
            cv=5,
            scoring='neg_mean_absolute_error'
        )
        gs_gcomp1_tr.fit(Train[features], Train[outcomes])
        xgb_lm1_tr.set_params(**gs_gcomp1_tr.best_params_)
        xgb_lm1_tr.fit(Train[features], Train[outcomes])
        models['xgb'] = xgb_lm1_tr

    # base learner 4
    if include_mlp:
        mlp_lm = MLPRegressor()
        param_grid_mlpr = {
            'hidden_layer_sizes': [ (100,), (150,), (100, 100), (150,150)],
            'alpha': [0.0001, 0.001, 0.01, 0.1],
        }
        gs_mlp_lm = GridSearchCV(estimator=mlp_lm,param_grid=param_grid_mlpr,cv=5,scoring="neg_mean_absolute_error")
        gs_mlp_lm.fit(Train[features],Train[outcomes])
        mlp_lm.set_params(**gs_mlp_lm.best_params_)
        mlp_lm.fit(Train[features],Train[outcomes])
        models['mlp'] = mlp_lm

    if include_gbr:
        gbr_lm = GradientBoostingRegressor()
        param_grid_gbr = {
        # 'n_estimators': [100, 200, 300],
        'learning_rate': [0.05, 0.1, 0.2],
        'max_depth': [15,20,25,35],
        #'min_samples_split': [2, 3, 4],
        #'min_samples_leaf': [1, 2, 3],
        #'max_features': [None, 'sqrt', 'log2']
        }
        gs_gbr_lm = GridSearchCV(estimator=gbr_lm,param_grid=param_grid_gbr,cv=5,scoring="neg_mean_absolute_error")
        gs_gbr_lm.fit(Train[features],Train[outcomes])
        gbr_lm.set_params(**gs_gbr_lm.best_params_)
        gbr_lm.fit(Train[features],Train[outcomes])
        models['gbr'] = gbr_lm

    # meta learner
    Q1fit = []
    if include_lm:
        Q1fit.append(lm_1.predict(Test[features]).ravel())
    if include_rg:
        Q1fit.append(rg_lm.predict(Test[features]).ravel())
    if include_xgb:
        Q1fit.append(xgb_lm1_tr.predict(Test[features]).ravel())
    if include_mlp:
        Q1fit.append(mlp_lm.predict(Test[features]).ravel())
    if include_gbr:
        Q1fit.append(gbr_lm.predict(Test[features]).ravel())

    lm_stack = linear_model.LinearRegression()
    lm_stack.fit(np.transpose(Q1fit),Test[outcomes])

    return models, lm_stack

def gcompu_predict(preddat,features,model,stack):
    y_pred_tr = {}
    for name, model in model.items():
        y_pred_tr[name] = model.predict(preddat[features]).ravel()
    Q_base_tr = np.transpose(np.array(list(y_pred_tr.values())))
    eta_tr = stack.predict(Q_base_tr)

    return eta_tr

def gcompu_function(Train,Test,features,outcomes,lm = True, rg = False, 
                    xgb = True, mlp = False,gbr = False):
    import warnings
    warnings.filterwarnings('ignore')
    # Train,Test = train_test_split(data,test_size=0.4,random_state=42)
    model_tr,stack_tr = gcompu_train_ensemble(Train,Test,features,outcomes,
                                              include_lm = lm, include_rg = rg, 
                                              include_xgb = xgb, include_mlp = mlp,include_gbr = gbr)
    model_ts,stack_ts = gcompu_train_ensemble(Test,Train,features,outcomes,
                                              include_lm = lm, include_rg = rg, 
                                              include_xgb = xgb, include_mlp = mlp,include_gbr = gbr)
    eta_tr=gcompu_predict(Test,features,model_tr,stack_tr)
    eta_ts=gcompu_predict(Train,features,model_ts,stack_ts)
    return model_tr,model_ts,stack_tr,stack_ts,eta_tr,eta_ts




In [ ]:
### This is to load the data for analysis and the covariates variable sets for analysis

# os.chdir('/Users/chengroup/Dropbox/MSM/ABCD')

abcd = pd.read_csv("ABCD5.0_flwup2_allnih.csv")
#from DR_function import dr,dr_boot
import pyreadr
robjects = pyreadr.read_r('ABCD5.0_varname_allnih.RData')
# Convert the R object to a Pandas DataFrame
covbase = robjects['covbase'].to_numpy().ravel()
output1 = robjects['outtp12_1'].to_numpy().ravel()
output2 = robjects['outtp12_2'].to_numpy().ravel()
covtp1 = robjects['covtp1'].to_numpy().ravel()
covtp2 = robjects['covtp2'].to_numpy().ravel()
trttp1 = robjects['trttp1'].to_numpy().ravel()
trttp2 = robjects['trttp2'].to_numpy().ravel()

# coding dummy variables for categorical variables (only consider race and income, education need to be discuss)
cat_cols = abcd.select_dtypes(include="object").columns.tolist()[2:]
dummy_df = pd.get_dummies(abcd[cat_cols],prefix_sep="",drop_first=True)
dummy_df.columns = dummy_df.columns.str.replace('_r', '', regex=True)
abcd_encoded = pd.concat([abcd.drop(cat_cols,axis=1),dummy_df],axis=1)
covbase = np.concatenate([[covbase[0]],covbase[4:],dummy_df.columns])
#abcd_encoded = abcd


# for two time points measure

In [ ]:
ps0feature = np.concatenate([covbase,covtp1])
ps1feature = np.concatenate([covbase,covtp1,covtp2, output1,trttp1])

baseps0_tr,baseps0_ts,cls_stack0_tr,cls_stack0_ts,enps0_tr,enps0_ts,enps0 = ps_function(abcd_encoded,ps0feature,trttp1,lg=False,el=False,rf=True,mlp = True)
    # = ps_predict(testdat,ps0feature,baseps0_tr,baseps0_ts,cls_stack0_tr,cls_stack0_ts)

baseps1_tr,baseps1_ts,cls_stack1_tr,cls_stack1_ts,enps1_tr,enps1_ts,enps1 = ps_function(abcd_encoded,ps1feature,trttp2,lg=False,el = False,rf=True,mlp = True)
    
IPW = (abcd_encoded[trttp1].values.ravel()/enps0+(1-abcd_encoded[trttp1].values.ravel())/(1-enps0))*(abcd_encoded[trttp2].values.ravel()/enps1+(1-abcd_encoded[trttp2].values.ravel())/(1-enps1))
msm_lm = linear_model.LinearRegression()
phi_msm_sum = []; phi_msm_ratio = []; phi_msm1 = []; phi_msm2 = []
for out2 in output2:
 # print(out2)
 msm_lm.fit(abcd_encoded[[trttp1[0],trttp2[0]]],abcd_encoded[out2],sample_weight=IPW)
 phi_msm_sum.append((np.sum(msm_lm.coef_))); phi_msm1.append(msm_lm.coef_[0]); phi_msm2.append(msm_lm.coef_[1])
 phi_msm_ratio.append(phi_msm_sum/(phi_msm_sum+msm_lm.intercept_))

In [ ]:
# Estimating funciton
def dr_abcd_boot(out1,out2,trt1 = trttp1,trt2 = trttp2,enps0_tr = enps0_tr, 
                 enps0_ts = enps0_ts,enps1_tr = enps1_tr,enps1_ts = enps1_ts,
                 data = abcd_encoded,covbase = covbase,
                 covtp1 = covtp1,covtp2 = covtp2,ps0feature = ps0feature,
                 ps1feature = ps1feature,B = 1000):
    # out1, out2, 
    out1 = [out1]; out2 = [out2];n = data.shape[0]
    trt1 = trt1[0]; trt2 = trt2[0]
    feature1 = np.concatenate([covbase,covtp1,[trt1]])
    feature2 = np.concatenate([covbase,covtp1,[trt1],[trt2],covtp2,out1])

    preddat11 = data.copy(); preddat11[trt1]=1;preddat11[trt2]=1
    preddat00 = data.copy(); preddat00[trt1]=0;preddat00[trt2]=0
    preddat10 = data.copy(); preddat10[trt1]=1;preddat10[trt2]=0
    preddat01 = data.copy(); preddat01[trt1]=0;preddat01[trt2]=1

    Train,Test = train_test_split(data,test_size=0.4,random_state=42)
    basemodel1_tr,basemodel1_ts,lm_stack1_tr,lm_stack1_ts,eta2_tr,eta2_ts = gcompu_function(Train,Test,feature2,out2,lm=False,mlp=True,gbr=True)

    # preddat1 = data.copy(); preddat1['t1']=1
    # preddat0 = data.copy(); preddat0['t1']=0
    nts = Test.shape[0];ntr = Train.shape[0]

    eta20_tr = gcompu_predict(Test.assign(**{trt2:0}),feature2,basemodel1_tr,lm_stack1_tr)
    eta20_ts = gcompu_predict(Train.assign(**{trt2:0}),feature2,basemodel1_ts,lm_stack1_ts)
    eta20_tr_series = pd.Series(eta20_tr.ravel(), index=Test.index)
    eta20_ts_series = pd.Series(eta20_ts.ravel(), index=Train.index)
    eta20 = pd.concat([eta20_tr_series, eta20_ts_series]).sort_index()
    basemodel20_tr,basemodel20_ts,lm_stack20_tr,lm_stack20_ts,eta200_tr,eta200_ts = gcompu_function(
        pd.concat([Train, pd.DataFrame(eta20_ts_series.values,index=Train.index, columns=['eta20'])], axis=1),
        pd.concat([Test, pd.DataFrame(eta20_tr_series.values,index=Test.index, columns=['eta20'])], axis=1),
        feature1,'eta20',lm=False,mlp=True,gbr=True)


    eta21_tr = gcompu_predict(Test.assign(**{trt2:1}),feature2,basemodel1_tr,lm_stack1_tr)
    eta21_ts = gcompu_predict(Train.assign(**{trt2:1}),feature2,basemodel1_ts,lm_stack1_ts)
    eta21_tr_series = pd.Series(eta21_tr.ravel(), index=Test.index)
    eta21_ts_series = pd.Series(eta21_ts.ravel(), index=Train.index)
    eta21 = pd.concat([eta21_tr_series, eta21_ts_series]).sort_index()
    basemodel21_tr,basemodel21_ts,lm_stack21_tr,lm_stack21_ts,eta211_tr,eta211_ts = gcompu_function(
        pd.concat([Train, pd.DataFrame(eta21_ts_series.values,index=Train.index, columns=['eta21'])], axis=1),
        pd.concat([Test, pd.DataFrame(eta21_tr_series.values,index=Test.index, columns=['eta21'])], axis=1),
        feature1,'eta21',lm=False,mlp=True,gbr=True)
    
    Q_00_tr = gcompu_predict(Test.assign(**{trt1:0, trt2:0}),feature1,basemodel20_tr,lm_stack20_tr)
    Q_00_ts= gcompu_predict(Train.assign(**{trt1:0, trt2:0}),feature1,basemodel20_ts,lm_stack20_ts)

    Q_11_tr = gcompu_predict(Test.assign(**{trt1:1, trt2:1}),feature1,basemodel21_tr,lm_stack21_tr)
    Q_11_ts= gcompu_predict(Train.assign(**{trt1:1, trt2:1}),feature1,basemodel21_ts,lm_stack21_ts)

    # Q_11_tr,Q_11_ts= gcompu_predict(preddat11,feature1,basemodel21_tr,basemodel21_ts,lm_stack21_tr,lm_stack21_ts)


    # phi_gcomp = (np.mean(Q_11-Q_00))
    eta10_tr = gcompu_predict(Test.assign(**{trt2:0}),feature1,basemodel20_tr,lm_stack20_tr)
    eta10_ts = gcompu_predict(Train.assign(**{trt2:0}),feature1,basemodel20_ts,lm_stack20_ts)
            
    # , = gcompu_predict(preddat0,feature1,basemodel20_tr,basemodel20_ts,lm_stack20_tr,lm_stack20_ts)
    eta11_tr = gcompu_predict(Test.assign(**{trt2:1}),feature1,basemodel21_tr,lm_stack21_tr)
    eta11_ts = gcompu_predict(Train.assign(**{trt2:1}),feature1,basemodel21_ts,lm_stack21_ts)
    # , = gcompu_predict(preddat1,feature1,basemodel21_tr,basemodel21_ts,lm_stack21_tr,lm_stack21_ts)

    eta0_00_tr = Q_00_tr; eta0_00_ts = Q_00_ts

    # preddat10 = data.copy(); preddat10['t0']=1;preddat10['t1']=0
    eta0_10_tr = gcompu_predict(Test.assign(**{trt1:1, trt2:0}),feature1,basemodel20_tr,lm_stack20_tr)
    eta0_10_ts = gcompu_predict(Train.assign(**{trt1:1, trt2:0}),feature1,basemodel20_ts,lm_stack20_ts)
    # , = gcompu_predict(preddat10,feature1,basemodel20_tr,basemodel20_ts,lm_stack20_tr,lm_stack20_ts)

    eta0_11_tr = Q_11_tr; eta0_11_ts = Q_11_ts

    # preddat01 = data.copy(); preddat01['t0']=0;preddat01['t1']=1
    eta0_01_tr = gcompu_predict(Test.assign(**{trt1:0, trt2:1}),feature1,basemodel21_tr,lm_stack21_tr)
    eta0_01_ts = gcompu_predict(Train.assign(**{trt1:0, trt2:1}),feature1,basemodel21_ts,lm_stack21_ts)
    # , = gcompu_predict(preddat01,feature1,basemodel21_tr,basemodel21_ts,lm_stack21_tr,lm_stack21_ts)


    ## linear equation for DR, subject to change based on MSM/other outcome equation of interest
    t0 = Test[trt1]; t1= Test[trt2]; y1 = Test[out2[0]] #y0 = testdat['y0']
    # enps0 = lmps0; enps1 = lmps1
    b0 = [4,2,2];b1 = [2,2,1];b2 = [2,1,2]
    f01_tr = ((t0/enps0_tr+(1-t0)/(1-enps0_tr))*(t1/enps1_tr+(1-t1)/(1-enps1_tr))*(y1-eta2_tr.ravel()))
    f02_tr = 1/((enps0_tr**t0)*(1-enps0_tr)**(1-t0))*(eta20_tr.ravel()-eta10_tr.ravel())+1/((enps0_tr**t0)*(1-enps0_tr)**(1-t0))*(eta21_tr.ravel()-eta11_tr.ravel())
    f03_tr = (eta0_00_tr+eta0_11_tr+eta0_10_tr.ravel()+eta0_01_tr.ravel())
    c0_tr = np.mean(f01_tr+f02_tr+f03_tr)
    
    f11_tr = (t0/enps0_tr+(1-t0)/(1-enps0_tr))*(t1/enps1_tr+(1-t1)/(1-enps1_tr))*(y1-eta2_tr.ravel())*t0
    f12_tr = t0/((enps0_tr**t0)*(1-enps0_tr)**(1-t0))*(eta20_tr.ravel()-eta10_tr.ravel())+t0/((enps0_tr**t0)*(1-enps0_tr)**(1-t0))*(eta21_tr.ravel()-eta11_tr.ravel())
    f13_tr = (eta0_11_tr+eta0_10_tr.ravel())
    c1_tr = np.mean(f11_tr+f12_tr+f13_tr)

    f21_tr = (t0/enps0_tr+(1-t0)/(1-enps0_tr))*(t1/enps1_tr+(1-t1)/(1-enps1_tr))*(y1-eta2_tr.ravel())*t1
    f22_tr = 1/((enps0_tr**t0)*(1-enps0_tr)**(1-t0))*(eta21_tr.ravel()-eta11_tr.ravel())
    f23_tr = (eta0_11_tr+eta0_01_tr.ravel())
    c2_tr = np.mean(f21_tr+f22_tr+f23_tr)

    coef_tr = np.linalg.solve(np.array([b0,b1,b2]),np.array([c0_tr,c1_tr,c2_tr]))

    # training set
    t0 = Train[trt1]; t1= Train[trt2]; y1 = Train[out2[0]]
    f01_ts = ((t0/enps0_ts+(1-t0)/(1-enps0_ts))*(t1/enps1_ts+(1-t1)/(1-enps1_ts))*(y1-eta2_ts.ravel()))
    f02_ts = 1/((enps0_ts**t0)*(1-enps0_ts)**(1-t0))*(eta20_ts.ravel()-eta10_ts.ravel())+1/((enps0_ts**t0)*(1-enps0_ts)**(1-t0))*(eta21_ts.ravel()-eta11_ts.ravel())
    f03_ts = (eta0_00_ts+eta0_11_ts+eta0_10_ts.ravel()+eta0_01_ts.ravel())
    c0_ts = np.mean(f01_ts+f02_ts+f03_ts)

    f11_ts = (t0/enps0_ts+(1-t0)/(1-enps0_ts))*(t1/enps1_ts+(1-t1)/(1-enps1_ts))*(y1-eta2_ts.ravel())*t0
    f12_ts = t0/((enps0_ts**t0)*(1-enps0_ts)**(1-t0))*(eta20_ts.ravel()-eta10_ts.ravel())+t0/((enps0_ts**t0)*(1-enps0_ts)**(1-t0))*(eta21_ts.ravel()-eta11_ts.ravel())
    f13_ts = (eta0_11_ts+eta0_10_ts.ravel())
    c1_ts = np.mean(f11_ts+f12_ts+f13_ts)

    f21_ts = (t0/enps0_ts+(1-t0)/(1-enps0_ts))*(t1/enps1_ts+(1-t1)/(1-enps1_ts))*(y1-eta2_ts.ravel())*t1
    f22_ts = 1/((enps0_ts**t0)*(1-enps0_ts)**(1-t0))*(eta21_ts.ravel()-eta11_ts.ravel())
    f23_ts = (eta0_11_ts+eta0_01_ts.ravel())
    c2_ts = np.mean(f21_ts+f22_ts+f23_ts)

    coef_ts = np.linalg.solve(np.array([b0,b1,b2]),np.array([c0_ts,c1_ts,c2_ts]))

    coef = np.linalg.solve(np.array([b0,b1,b2]),np.array([(c0_ts+c0_tr)/2,(c1_ts+c1_tr)/2,(c2_ts+c2_tr)/2]))
    phi_dr = coef[1]+coef[2]; beta1 = coef[1]; beta2 = coef[2]

    Bmat = np.array([[-4,-2,-2],
                     [-2,-2,-1],
                     [-2,-1,-2]])
    Bmatinv = np.linalg.inv(Bmat)
    phi1 = np.concatenate([f01_tr, f01_ts])+np.concatenate([f02_tr,f02_ts])+np.concatenate([f03_tr,f03_ts])-4*(coef[0])-2*(coef[1])-2*(coef[2])
    phi2 = np.concatenate([f11_tr,f11_ts])+np.concatenate([f12_tr,f12_ts])+np.concatenate([f13_tr,f13_ts])-2*(coef[0])-2*(coef[1])-(coef[2])
    phi3 = np.concatenate([f21_tr,f21_ts])+np.concatenate([f22_tr,f22_ts])+np.concatenate([f23_tr,f23_ts])-2*(coef[0])-(coef[1])-2*(coef[2])

    Phi = np.array([phi1,phi2,phi3])
    Fmat = np.zeros((3,3))
    for i in range(n):
        phi_vector = Phi[:,i]
        outer_product = np.outer(phi_vector,phi_vector)
        Fmat += outer_product
    Fmat /= n
    Vmat = (Bmatinv @ Fmat @ Bmatinv.T)/(n)
    dr_se  = np.sqrt(Vmat[1,1]+Vmat[2,2]+2*Vmat[1,2])
    drb1_se = np.sqrt((Vmat[1,1])); drb2_se = np.sqrt((Vmat[2,2]))
    # dr_se  = np.sqrt(((Vmat_dr[1,1]+Vmat_dr[2,2]+2*Vmat_dr[1,2])+(Vmat_ds[1,1]+Vmat_ds[2,2]+2*Vmat_ds[1,2]))/2)
    # drb1_se = np.sqrt((Vmat_dr[1,1]+Vmat_ds[1,1])/2); drb2_se = np.sqrt((Vmat_dr[2,2]+Vmat_ds[2,2])/2)

    phi1_dr = (f01_tr)+(f02_tr)+(f03_tr)-4*(coef_tr[0])-2*(coef_tr[1])-2*(coef_tr[2])
    phi2_dr = (f11_tr)+(f12_tr)+(f13_tr)-2*(coef_tr[0])-2*(coef_tr[1])-(coef_tr[2])
    phi3_dr = (f21_tr)+(f22_tr)+(f23_tr)-2*(coef_tr[0])-(coef_tr[1])-2*(coef_tr[2])
    phi1_ds = (f01_ts)+(f02_ts)+(f03_ts)-4*(coef_ts[0])-2*(coef_ts[1])-2*(coef_ts[2])
    phi2_ds = (f11_ts)+(f12_ts)+(f13_ts)-2*(coef_ts[0])-2*(coef_ts[1])-(coef_ts[2])
    phi3_ds = (f21_ts)+(f22_ts)+(f23_ts)-2*(coef_ts[0])-(coef_ts[1])-2*(coef_ts[2])
    Phi = np.column_stack([np.concatenate([phi1_dr,phi1_ds]),np.concatenate([phi2_dr,phi2_ds]),np.concatenate([phi3_dr,phi3_ds])])
    Sigma = np.dot(Phi.T,Phi)/(2*n)
    dr_se_chern = np.sqrt((Sigma[1,1]+Sigma[2,2]+2*Sigma[1,2])/(2*n))


     ## G-comp with LR
    lm1 = linear_model.LinearRegression()
    lm1.fit(data[feature2],data[out2])
    Q11 = lm1.predict(preddat11[feature2])
    Q00 = lm1.predict(preddat00[feature2])

    lm0_1 = linear_model.LinearRegression()
    lm0_1.fit(data[feature1],Q11)
    lm0_0 = linear_model.LinearRegression()
    lm0_0.fit(data[feature1],Q00)
    phi_gcomp_lm = (np.mean(lm0_1.predict(preddat11[feature1])-lm0_0.predict(preddat00[feature1])))



    gcomplm_boot = [];   
    msmlm_boot = []; msmlm_b1boot = []; msmlm_b2boot = []
    # gcomp_boot = []; gcomp_b1boot = []; gcomp_b2boot = []
    for b in range(B):
        np.random.seed(233+b)
        sampleid = np.random.choice(range(1, n+1), size=n, replace=True)
        dat_boot = data.iloc[sampleid-1, :]
        bootpreddat11 = dat_boot.copy(); bootpreddat11[trt1]=1;bootpreddat11[trt2]=1
        bootpreddat00 = dat_boot.copy(); bootpreddat00[trt1]=0;bootpreddat00[trt2]=0
        bootpreddat10 = dat_boot.copy(); bootpreddat10[trt1]=1;bootpreddat10[trt2]=0
        bootpreddat01 = dat_boot.copy(); bootpreddat01[trt1]=0;bootpreddat01[trt2]=1

        pred_boot1 = dat_boot.copy(); pred_boot1[trt2]=1
        pred_boot0 = dat_boot.copy(); pred_boot0[trt2]=0

        ## G-comp with LR
        lm1boot = linear_model.LinearRegression()
        lm1boot.fit(dat_boot[feature2],dat_boot[out2])
        Q11boot = lm1boot.predict(bootpreddat11[feature2])
        Q00boot = lm1boot.predict(bootpreddat00[feature2])

        lm0_1boot = linear_model.LinearRegression()
        lm0_1boot.fit(dat_boot[feature1],Q11boot)
        lm0_0boot = linear_model.LinearRegression()
        lm0_0boot.fit(dat_boot[feature1],Q00boot)
        gcomplm_boot.append(np.mean(lm0_1boot.predict(bootpreddat11[feature1])-lm0_0boot.predict(bootpreddat00[feature1])))

        ## MSM with LR
        lgboot0 = linear_model.LogisticRegression().fit(dat_boot[ps0feature],dat_boot[trt1])
        lmpsboot0 = np.clip(lgboot0.predict_proba(dat_boot[ps0feature])[:,1].ravel(),0.01,0.99)
        lgboot1 = linear_model.LogisticRegression().fit(dat_boot[ps1feature],dat_boot[trt2])
        lmpsboot1 = np.clip(lgboot1.predict_proba(dat_boot[ps1feature])[:,1].ravel(),0.01,0.99)
        lmPboot = (dat_boot[trt1].values.ravel()/lmpsboot0+(1-dat_boot[trt1].values.ravel())/(1-lmpsboot0))*(dat_boot[trt2].values.ravel()/lmpsboot1+(1-dat_boot[trt2].values.ravel())/(1-lmpsboot1))
        msm_lgboot = linear_model.LinearRegression().fit(dat_boot[[trt1,trt2]],dat_boot[out2],sample_weight=lmPboot)
        msmlm_boot.append(np.sum(msm_lgboot.coef_))
        msmlm_b1boot.append(msm_lgboot.coef_[0][0]); msmlm_b2boot.append(msm_lgboot.coef_[0][1])

    gcomplm_se = np.std(gcomplm_boot)
    msmlm_se = np.std(msmlm_boot); msmlmb1_se = np.std(msmlm_b1boot); msmlmb2_se = np.std(msmlm_b2boot)

       

    return  phi_dr, beta1, beta2,phi_gcomp_lm, dr_se, drb2_se,drb1_se, gcomplm_se,msmlm_se,msmlmb1_se,msmlmb2_se,dr_se_chern

In [ ]:
from functools import partial
start_time = time.time()
#np.random.seed(233)
with multiprocess.Pool(processes=30) as pool:
    #np.random.seed(233)
    result_abcd = pool.starmap(partial(dr_abcd_boot,trt1 = trttp1,trt2 = trttp2,enps0_tr = enps0_tr, 
                                       enps0_ts = enps0_ts,enps1_tr = enps1_tr,enps1_ts = enps1_ts,
                                       data = abcd_encoded,covbase = covbase,
                                       covtp1 = covtp1,covtp2 = covtp2,ps0feature = ps0feature, 
                                       ps1feature = ps1feature), zip(output1,output2))
    pool.close(); pool.join()

end_time = time.time()
(end_time-start_time)

# For three time point

In [ ]:

abcd3 = pd.read_csv("ABCD5.0_medianimpu_three.csv")
#from DR_function import dr,dr_boot
import pyreadr
robjects = pyreadr.read_r('ABCD5.0_varname_allnih.RData')
# robjects3 = pyreadr.read_r('ABCD5.0_varname3.RData')
# Convert the R object to a Pandas DataFrame
covbase = robjects['covbase'].to_numpy().ravel()
output1 = robjects['outtp123_1'].to_numpy().ravel()
output2 = robjects['outtp123_2'].to_numpy().ravel()
output3 = robjects['outtp123_3'].to_numpy().ravel()
covtp1 = robjects['covtp1'].to_numpy().ravel()
covtp2 = robjects['covtp2'].to_numpy().ravel()
covtp3 = robjects['covtp3'].to_numpy().ravel()
trttp1 = robjects['trttp1'].to_numpy().ravel()
trttp2 = robjects['trttp2'].to_numpy().ravel()
trttp3 = robjects['trttp3'].to_numpy().ravel()

# coding dummy variables for categorical variables (only consider race and income, education need to be discuss)
cat_cols3 = abcd3.select_dtypes(include="object").columns.tolist()[2:]
dummy_df3 = pd.get_dummies(abcd3[cat_cols3],prefix_sep="",drop_first=True)
dummy_df3.columns = dummy_df3.columns.str.replace('_r', '', regex=True)
abcd_encoded = pd.concat([abcd3.drop(cat_cols3,axis=1),dummy_df3],axis=1)
covbase = np.concatenate([[covbase[0]],covbase[4:],dummy_df3.columns])
#abcd_encoded = abcds

# from simulation_function_abcd import gcompu_train_ensemble,ps_train_ensemble

In [ ]:
# with common propensity score
ps0feature = np.concatenate([covbase,covtp1])
ps1feature = np.concatenate([covbase,covtp1,covtp2, output1,trttp1])
ps2feature = np.concatenate([covbase,covtp1,covtp2,covtp3, output1,output2,trttp1,trttp2])

baseps0_tr,baseps0_ts,cls_stack0_tr,cls_stack0_ts,enps0_tr,enps0_ts,enps0 = ps_function(abcd_encoded,ps0feature,trttp1,lg=False,mlp=True,el=False,rf=True,nb=True)

baseps1_tr,baseps1_ts,cls_stack1_tr,cls_stack1_ts,enps1_tr,enps1_ts,enps1 = ps_function(abcd_encoded,ps1feature,trttp2,lg=False,mlp=True,el=False,rf=True,nb=True)

baseps2_tr,baseps2_ts,cls_stack2_tr,cls_stack2_ts,enps2_tr,enps2_ts,enps2 = ps_function(abcd_encoded,ps2feature,trttp3,lg=False,mlp=True,el=False,rf=True,nb=True)


In [ ]:

def dr_abcd_boot3(out1,out2,out3,trt1 = trttp1,trt2 = trttp2,enps0_tr = enps0_tr, 
                 enps0_ts = enps0_ts,enps1_tr = enps1_tr,enps1_ts = enps1_ts,
                 trt3 = trttp3, covtp3 = covtp3, enps2_tr = enps2_tr,enps2_ts = enps2_ts,
                 data = abcd_encoded,covbase = covbase,ps2feature = ps2feature,
                 covtp1 = covtp1,covtp2 = covtp2,ps0feature = ps0feature,
                 ps1feature = ps1feature,B = 1000):
    # out1, out2, 
    out1 = [out1]; out2 = [out2];n = data.shape[0];out3 = [out3]
    trt1 = trt1[0]; trt2 = trt2[0];trt3 = trt3[0]
    feature1 = np.concatenate([covbase,covtp1,[trt1]])
    feature2 = np.concatenate([covbase,covtp1,[trt1],[trt2],covtp2,out1])
    feature3 = np.concatenate([covbase,covtp1,[trt1],[trt2],covtp2,out1,out2,covtp3,[trt3]])

    preddat11 = data.copy(); preddat11[trt1]=1;preddat11[trt2]=1
    preddat00 = data.copy(); preddat00[trt1]=0;preddat00[trt2]=0
    preddat10 = data.copy(); preddat10[trt1]=1;preddat10[trt2]=0
    preddat01 = data.copy(); preddat01[trt1]=0;preddat01[trt2]=1

    Train,Test = train_test_split(data,test_size=0.4,random_state=42)
    basemodel1_tr,basemodel1_ts,lm_stack1_tr,lm_stack1_ts,eta3_tr,eta3_ts = gcompu_function(Train,Test,feature3,out3,lm=False,mlp=True,gbr=True)

    # preddat1 = data.copy(); preddat1['t1']=1
    # preddat0 = data.copy(); preddat0['t1']=0
    nts = Test.shape[0];ntr = Train.shape[0]

    eta30_tr = gcompu_predict(Test.assign(**{trt3:0}),feature3,basemodel1_tr,lm_stack1_tr)
    eta30_ts = gcompu_predict(Train.assign(**{trt3:0}),feature3,basemodel1_ts,lm_stack1_ts)
    eta30_tr_series = pd.Series(eta30_tr.ravel(), index=Test.index)
    eta30_ts_series = pd.Series(eta30_ts.ravel(), index=Train.index)
    eta30 = pd.concat([eta30_tr_series, eta30_ts_series]).sort_index()
    basemodel30_tr,basemodel30_ts,lm_stack30_tr,lm_stack30_ts,eta300_tr,eta300_ts = gcompu_function(
        pd.concat([Train, pd.DataFrame(eta30_ts_series.values,index=Train.index, columns=['eta30'])], axis=1),
        pd.concat([Test, pd.DataFrame(eta30_tr_series.values,index=Test.index, columns=['eta30'])], axis=1),
        feature2,'eta30',lm=False,mlp=True,gbr=True)


    eta31_tr = gcompu_predict(Test.assign(**{trt3:1}),feature3,basemodel1_tr,lm_stack1_tr)
    eta31_ts = gcompu_predict(Train.assign(**{trt3:1}),feature3,basemodel1_ts,lm_stack1_ts)
    eta31_tr_series = pd.Series(eta31_tr.ravel(), index=Test.index)
    eta31_ts_series = pd.Series(eta31_ts.ravel(), index=Train.index)
    eta31 = pd.concat([eta31_tr_series, eta31_ts_series]).sort_index()
    basemodel31_tr,basemodel31_ts,lm_stack31_tr,lm_stack31_ts,eta311_tr,eta311_ts = gcompu_function(
        pd.concat([Train, pd.DataFrame(eta31_ts_series.values,index=Train.index, columns=['eta31'])], axis=1),
        pd.concat([Test, pd.DataFrame(eta31_tr_series.values,index=Test.index, columns=['eta31'])], axis=1),
        feature2,'eta31',lm=False,mlp=True,gbr=True)
    
    eta20_tr = gcompu_predict(Test.assign(**{trt3:0}),feature2,basemodel30_tr,lm_stack30_tr)
    eta20_ts = gcompu_predict(Train.assign(**{trt3:0}),feature2,basemodel30_ts,lm_stack30_ts)

    eta21_tr = gcompu_predict(Test.assign(**{trt3:1}),feature2,basemodel31_tr,lm_stack31_tr)
    eta21_ts = gcompu_predict(Train.assign(**{trt3:1}),feature2,basemodel31_ts,lm_stack31_ts)

    eta200_tr = gcompu_predict(Test.assign(**{trt2:0,trt3:0}),feature2,basemodel30_tr,lm_stack30_tr)
    eta200_ts = gcompu_predict(Train.assign(**{trt2:0,trt3:0}),feature2,basemodel30_ts,lm_stack30_ts)
    eta200_tr_series = pd.Series(eta200_tr.ravel(), index=Test.index)
    eta200_ts_series = pd.Series(eta200_ts.ravel(), index=Train.index)
    eta200 = pd.concat([eta200_tr_series, eta200_ts_series]).sort_index()
    basemodel200_tr,basemodel200_ts,lm_stack200_tr,lm_stack200_ts,eta2000_tr,eta2000_ts = gcompu_function(
        pd.concat([Train, pd.DataFrame(eta200_ts_series.values,index=Train.index, columns=['eta200'])], axis=1),
        pd.concat([Test, pd.DataFrame(eta200_tr_series.values,index=Test.index, columns=['eta200'])], axis=1),
        feature1,'eta200',lm=True,mlp=True,gbr=True)
    
    eta211_tr = gcompu_predict(Test.assign(**{trt2:1,trt3:1}),feature2,basemodel31_tr,lm_stack31_tr)
    eta211_ts = gcompu_predict(Train.assign(**{trt2:1,trt3:1}),feature2,basemodel31_ts,lm_stack31_ts)
    eta211_tr_series = pd.Series(eta211_tr.ravel(), index=Test.index)
    eta211_ts_series = pd.Series(eta211_ts.ravel(), index=Train.index)
    eta211 = pd.concat([eta211_tr_series, eta211_ts_series]).sort_index()
    basemodel211_tr,basemodel211_ts,lm_stack211_tr,lm_stack211_ts,eta2110_tr,eta2110_ts = gcompu_function(
        pd.concat([Train, pd.DataFrame(eta211_ts_series.values,index=Train.index, columns=['eta211'])], axis=1),
        pd.concat([Test, pd.DataFrame(eta211_tr_series.values,index=Test.index, columns=['eta211'])], axis=1),
        feature1,'eta211',lm=True,mlp=True,gbr=True)


    eta201_tr = gcompu_predict(Test.assign(**{trt2:0,trt3:1}),feature2,basemodel31_tr,lm_stack31_tr)
    eta201_ts = gcompu_predict(Train.assign(**{trt2:0,trt3:1}),feature2,basemodel31_ts,lm_stack31_ts)
    eta201_tr_series = pd.Series(eta201_tr.ravel(), index=Test.index)
    eta201_ts_series = pd.Series(eta201_ts.ravel(), index=Train.index)
    eta201 = pd.concat([eta201_tr_series, eta201_ts_series]).sort_index()
    basemodel201_tr,basemodel201_ts,lm_stack201_tr,lm_stack201_ts,eta2010_tr,eta2010_ts = gcompu_function(
        pd.concat([Train, pd.DataFrame(eta201_ts_series.values,index=Train.index, columns=['eta201'])], axis=1),
        pd.concat([Test, pd.DataFrame(eta201_tr_series.values,index=Test.index, columns=['eta201'])], axis=1),
        feature1,'eta201',lm=True,mlp=True,gbr=True)
    

    eta210_tr = gcompu_predict(Test.assign(**{trt2:1,trt3:0}),feature2,basemodel30_tr,lm_stack30_tr)
    eta210_ts = gcompu_predict(Train.assign(**{trt2:1,trt3:0}),feature2,basemodel30_ts,lm_stack30_ts)
    eta210_tr_series = pd.Series(eta210_tr.ravel(), index=Test.index)
    eta210_ts_series = pd.Series(eta210_ts.ravel(), index=Train.index)
    eta210 = pd.concat([eta210_tr_series, eta210_ts_series]).sort_index()
    basemodel210_tr,basemodel210_ts,lm_stack210_tr,lm_stack210_ts,eta2100_tr,eta2100_ts = gcompu_function(
        pd.concat([Train, pd.DataFrame(eta210_ts_series.values,index=Train.index, columns=['eta210'])], axis=1),
        pd.concat([Test, pd.DataFrame(eta210_tr_series.values,index=Test.index, columns=['eta210'])], axis=1),
        feature1,'eta210',lm=True,mlp=True,gbr=True)


    eta1_11_tr = gcompu_predict(Test.assign(**{trt2:1,trt3:1}),feature1,basemodel211_tr,lm_stack211_tr)
    eta1_11_ts = gcompu_predict(Train.assign(**{trt2:1,trt3:1}),feature1,basemodel211_ts,lm_stack211_ts)
    eta1_00_tr = gcompu_predict(Test.assign(**{trt2:0,trt3:0}),feature1,basemodel200_tr,lm_stack200_tr)
    eta1_00_ts = gcompu_predict(Train.assign(**{trt2:0,trt3:0}),feature1,basemodel200_ts,lm_stack200_ts)
    eta1_10_tr = gcompu_predict(Test.assign(**{trt2:1,trt3:0}),feature1,basemodel210_tr,lm_stack210_tr)
    eta1_10_ts = gcompu_predict(Train.assign(**{trt2:1,trt3:0}),feature1,basemodel210_ts,lm_stack210_ts)
    eta1_01_tr = gcompu_predict(Test.assign(**{trt2:0,trt3:1}),feature1,basemodel201_tr,lm_stack201_tr)
    eta1_01_ts = gcompu_predict(Train.assign(**{trt2:0,trt3:1}),feature1,basemodel201_ts,lm_stack201_ts)
    
    eta1_000_tr = gcompu_predict(Test.assign(**{trt1:0,trt2:0,trt3:0}),feature1,basemodel200_tr,lm_stack200_tr)
    eta1_000_ts = gcompu_predict(Train.assign(**{trt1:0,trt2:0,trt3:0}),feature1,basemodel200_ts,lm_stack200_ts)
    eta1_100_tr = gcompu_predict(Test.assign(**{trt1:1,trt2:0,trt3:0}),feature1,basemodel200_tr,lm_stack200_tr)
    eta1_100_ts = gcompu_predict(Train.assign(**{trt1:1,trt2:0,trt3:0}),feature1,basemodel200_ts,lm_stack200_ts)
    eta1_010_tr = gcompu_predict(Test.assign(**{trt1:0,trt2:1,trt3:0}),feature1,basemodel210_tr,lm_stack210_tr)
    eta1_010_ts = gcompu_predict(Train.assign(**{trt1:0,trt2:1,trt3:0}),feature1,basemodel210_ts,lm_stack210_ts)
    eta1_110_tr = gcompu_predict(Test.assign(**{trt1:1,trt2:1,trt3:0}),feature1,basemodel210_tr,lm_stack210_tr)
    eta1_110_ts = gcompu_predict(Train.assign(**{trt1:1,trt2:1,trt3:0}),feature1,basemodel210_ts,lm_stack210_ts)
    eta1_001_tr = gcompu_predict(Test.assign(**{trt1:0,trt2:0,trt3:1}),feature1,basemodel201_tr,lm_stack201_tr)
    eta1_001_ts = gcompu_predict(Train.assign(**{trt1:1,trt2:0,trt3:1}),feature1,basemodel201_ts,lm_stack201_ts)
    eta1_101_tr = gcompu_predict(Test.assign(**{trt1:1,trt2:0,trt3:1}),feature1,basemodel201_tr,lm_stack201_tr)
    eta1_101_ts = gcompu_predict(Train.assign(**{trt1:1,trt2:0,trt3:1}),feature1,basemodel201_ts,lm_stack201_ts)
    eta1_011_tr = gcompu_predict(Test.assign(**{trt1:0,trt2:1,trt3:1}),feature1,basemodel211_tr,lm_stack211_tr)
    eta1_011_ts = gcompu_predict(Train.assign(**{trt1:0,trt2:1,trt3:1}),feature1,basemodel211_ts,lm_stack211_ts)
    eta1_111_tr = gcompu_predict(Test.assign(**{trt1:1,trt2:1,trt3:1}),feature1,basemodel211_tr,lm_stack211_tr)
    eta1_111_ts = gcompu_predict(Train.assign(**{trt1:1,trt2:1,trt3:1}),feature1,basemodel211_ts,lm_stack211_ts)

    ## linear equation for DR, subject to change based on MSM/other outcome equation of interest
    t1 = Test[trt1]; t2= Test[trt2];t3 = Test[trt3]; y1 = Test[out3[0]] #y0 = testdat['y0']
    # enps0 = lmps0; enps1 = lmps1
    b0 = [8,4,4,4];b1 = [4,4,2,2];b2 = [4,2,4,2];b3 = [4,2,2,4]
    f01_tr = (t1/enps0_tr+(1-t1)/(1-enps0_tr))*(t2/enps1_tr+(1-t2)/(1-enps1_tr))*(t3/enps2_tr+(1-t3)/(1-enps2_tr))*(y1-eta3_tr.ravel())
    f02_tr = (t1/enps0_tr+(1-t1)/(1-enps0_tr))*(t2/enps1_tr+(1-t2)/(1-enps1_tr))*(eta30_tr.ravel()-eta20_tr.ravel())+(t1/enps0_tr+(1-t1)/(1-enps0_tr))*(t2/enps1_tr+(1-t2)/(1-enps1_tr))*(eta31_tr.ravel()-eta21_tr.ravel())
    f03_tr = (t1/enps0_tr+(1-t1)/(1-enps0_tr))*(eta200_tr-eta1_00_tr)+(t1/enps0_tr+(1-t1)/(1-enps0_tr))*(eta210_tr-eta1_10_tr)+(t1/enps0_tr+(1-t1)/(1-enps0_tr))*(eta201_tr-eta1_01_tr)+(t1/enps0_tr+(1-t1)/(1-enps0_tr))*(eta211_tr-eta1_11_tr)
    f04_tr = (eta1_000_tr+eta1_100_tr+eta1_010_tr+eta1_001_tr+eta1_110_tr+eta1_101_tr+eta1_011_tr+eta1_111_tr)
    c0_tr = np.mean(f01_tr+f02_tr+f03_tr+f04_tr)
    # c0_tr = np.mean(f01_tr+f02_tr+f03_tr)
    
    f11_tr = f01_tr*t1
    f12_tr = f02_tr*t1
    f13_tr = f03_tr*t1
    f14_tr = eta1_100_tr+eta1_110_tr+eta1_101_tr+eta1_111_tr
    c1_tr = np.mean(f11_tr+f12_tr+f13_tr+f14_tr)

    f21_tr = f01_tr*t2
    f22_tr = f02_tr*t2
    f23_tr = (t1/enps0_tr+(1-t1)/(1-enps0_tr))*(eta210_tr-eta1_10_tr)+(t1/enps0_tr+(1-t1)/(1-enps0_tr))*(eta211_tr-eta1_11_tr)
    f24_tr = eta1_010_tr+eta1_110_tr+eta1_011_tr+eta1_111_tr
    c2_tr = np.mean(f21_tr+f22_tr+f23_tr+f24_tr)

    f31_tr = f01_tr*t3
    f32_tr = (t1/enps0_tr+(1-t1)/(1-enps0_tr))*(t2/enps1_tr+(1-t2)/(1-enps1_tr))*(eta31_tr.ravel()-eta21_tr.ravel())
    f33_tr = (t1/enps0_tr+(1-t1)/(1-enps0_tr))*(eta201_tr-eta1_01_tr)+(t1/enps0_tr+(1-t1)/(1-enps0_tr))*(eta211_tr-eta1_11_tr)
    f34_tr = eta1_001_tr+eta1_011_tr+eta1_101_tr+eta1_111_tr
    c3_tr = np.mean(f31_tr+f32_tr+f33_tr+f34_tr)

    coef_tr = np.linalg.solve(np.array([b0,b1,b2,b3]),np.array([c0_tr,c1_tr,c2_tr,c3_tr]))

    # training set
    t1 = Train[trt1]; t2= Train[trt2];t3 = Train[trt3]; y1 = Train[out3[0]] #y0 = testdat['y0']
    f01_ts = (t1/enps0_ts+(1-t1)/(1-enps0_ts))*(t2/enps1_ts+(1-t2)/(1-enps1_ts))*(t3/enps2_ts+(1-t3)/(1-enps2_ts))*(y1-eta3_ts.ravel())
    f02_ts = (t1/enps0_ts+(1-t1)/(1-enps0_ts))*(t2/enps1_ts+(1-t2)/(1-enps1_ts))*(eta30_ts.ravel()-eta20_ts.ravel())+(t1/enps0_ts+(1-t1)/(1-enps0_ts))*(t2/enps1_ts+(1-t2)/(1-enps1_ts))*(eta31_ts.ravel()-eta21_ts.ravel())
    f03_ts = (t1/enps0_ts+(1-t1)/(1-enps0_ts))*(eta200_ts-eta1_00_ts)+(t1/enps0_ts+(1-t1)/(1-enps0_ts))*(eta210_ts-eta1_10_ts)+(t1/enps0_ts+(1-t1)/(1-enps0_ts))*(eta201_ts-eta1_01_ts)+(t1/enps0_ts+(1-t1)/(1-enps0_ts))*(eta211_ts-eta1_11_ts)
    f04_ts = (eta1_000_ts+eta1_100_ts+eta1_010_ts+eta1_001_ts+eta1_110_ts+eta1_101_ts+eta1_011_ts+eta1_111_ts)
    c0_ts = np.mean(f01_ts+f02_ts+f03_ts+f04_ts)

    f11_ts = f01_ts*t1
    f12_ts = f02_ts*t1
    f13_ts = f03_ts*t1
    f14_ts = eta1_100_ts+eta1_110_ts+eta1_101_ts+eta1_111_ts
    c1_ts = np.mean(f11_ts+f12_ts+f13_ts+f14_ts)

    f21_ts = f01_ts*t2
    f22_ts = f02_ts*t2
    f23_ts = (t1/enps0_ts+(1-t1)/(1-enps0_ts))*(eta210_ts-eta1_10_ts)+(t1/enps0_ts+(1-t1)/(1-enps0_ts))*(eta211_ts-eta1_11_ts)
    f24_ts = eta1_010_ts+eta1_110_ts+eta1_011_ts+eta1_111_ts
    c2_ts = np.mean(f21_ts+f22_ts+f23_ts+f24_ts)

    f31_ts = f01_ts*t3
    f32_ts = (t1/enps0_ts+(1-t1)/(1-enps0_ts))*(t2/enps1_ts+(1-t2)/(1-enps1_ts))*(eta31_ts.ravel()-eta21_ts.ravel())
    f33_ts = (t1/enps0_ts+(1-t1)/(1-enps0_ts))*(eta201_ts-eta1_01_ts)+(t1/enps0_ts+(1-t1)/(1-enps0_ts))*(eta211_ts-eta1_11_ts)
    f34_ts = eta1_001_ts+eta1_011_ts+eta1_101_ts+eta1_111_ts
    c3_ts = np.mean(f31_ts+f32_ts+f33_ts+f34_ts)

    coef_ts = np.linalg.solve(np.array([b0,b1,b2,b3]),np.array([c0_ts,c1_ts,c2_ts,c3_ts]))

    coef = np.linalg.solve(np.array([b0,b1,b2,b3]),np.array([(c0_ts+c0_tr)/2,(c1_ts+c1_tr)/2,(c2_ts+c2_tr)/2,(c3_ts+c3_tr)/2]))
    phi_dr = coef[1]+coef[2]; beta1 = coef[1]; beta2 = coef[2]; beta3 = coef[3]
# [8,4,4,4];b1 = [4,4,2,2];b2 = [4,2,4,2];b3 = [4,2,2,4]
    Bmat = np.array([[-8,-4,-4,-4],
                     [-4,-4,-2,-2],
                     [-4,-2,-4,-2],
                     [-4,-2,-2,-4]])
    Bmatinv = np.linalg.inv(Bmat)
    phi1 = np.concatenate([f01_tr, f01_ts])+np.concatenate([f02_tr,f02_ts])+np.concatenate([f03_tr,f03_ts])-8*(coef[0])-4*(coef[1])-4*(coef[2])-4*(coef[3])
    phi2 = np.concatenate([f11_tr,f11_ts])+np.concatenate([f12_tr,f12_ts])+np.concatenate([f13_tr,f13_ts])-4*(coef[0])-4*(coef[1])-2*(coef[2])-2*(coef[3])
    phi3 = np.concatenate([f21_tr,f21_ts])+np.concatenate([f22_tr,f22_ts])+np.concatenate([f23_tr,f23_ts])-4*(coef[0])-2*(coef[1])-4*(coef[2])-2*(coef[3])
    phi4 = np.concatenate([f31_tr,f31_ts])+np.concatenate([f32_tr,f32_ts])+np.concatenate([f33_tr,f33_ts])-4*(coef[0])-2*(coef[1])-2*(coef[2])-4*(coef[3])

    Phi = np.array([phi1,phi2,phi3,phi4])
    Fmat = np.zeros((4,4))
    for i in range(n):
        phi_vector = Phi[:,i]
        outer_product = np.outer(phi_vector,phi_vector)
        Fmat += outer_product
    Fmat /= n
    Vmat = (Bmatinv @ Fmat @ Bmatinv.T)/(n)
    dr_se  = np.sqrt(Vmat[1,1]+Vmat[2,2]+Vmat[3,3]+2*Vmat[1,2]+2*Vmat[1,3]+2*Vmat[2,3])
    drb1_se = np.sqrt((Vmat[1,1])); drb2_se = np.sqrt((Vmat[2,2]));drb3_se = np.sqrt(Vmat[3,3])
    # dr_se  = np.sqrt(((Vmat_dr[1,1]+Vmat_dr[2,2]+2*Vmat_dr[1,2])+(Vmat_ds[1,1]+Vmat_ds[2,2]+2*Vmat_ds[1,2]))/2)
    # drb1_se = np.sqrt((Vmat_dr[1,1]+Vmat_ds[1,1])/2); drb2_se = np.sqrt((Vmat_dr[2,2]+Vmat_ds[2,2])/2)

    # phi1_dr = (f01_tr)+(f02_tr)+(f03_tr)-4*(coef_tr[0])-2*(coef_tr[1])-2*(coef_tr[2])
    # phi2_dr = (f11_tr)+(f12_tr)+(f13_tr)-2*(coef_tr[0])-2*(coef_tr[1])-(coef_tr[2])
    # phi3_dr = (f21_tr)+(f22_tr)+(f23_tr)-2*(coef_tr[0])-(coef_tr[1])-2*(coef_tr[2])
    # phi1_ds = (f01_ts)+(f02_ts)+(f03_ts)-4*(coef_ts[0])-2*(coef_ts[1])-2*(coef_ts[2])
    # phi2_ds = (f11_ts)+(f12_ts)+(f13_ts)-2*(coef_ts[0])-2*(coef_ts[1])-(coef_ts[2])
    # phi3_ds = (f21_ts)+(f22_ts)+(f23_ts)-2*(coef_ts[0])-(coef_ts[1])-2*(coef_ts[2])
    # Phi = np.column_stack([np.concatenate([phi1_dr,phi1_ds]),np.concatenate([phi2_dr,phi2_ds]),np.concatenate([phi3_dr,phi3_ds])])
    # Sigma = np.dot(Phi.T,Phi)/(2*n)
    # dr_se_chern = np.sqrt((Sigma[1,1]+Sigma[2,2]+2*Sigma[1,2])/(2*n))

    preddat111 = data.copy(); preddat111[trt1]=1;preddat111[trt2]=1;preddat111[trt3] = 1
    preddat000 = data.copy(); preddat000[trt1]=0;preddat000[trt2]=0;preddat000[trt3] = 0
     ## G-comp with LR
    lm1 = linear_model.LinearRegression()
    lm1.fit(data[feature3],data[out3])
    Q111 = lm1.predict(preddat111[feature3])
    Q000 = lm1.predict(preddat000[feature3])

    lm0_1 = linear_model.LinearRegression()
    lm0_1.fit(data[feature2],Q111)
    lm0_0 = linear_model.LinearRegression()
    lm0_0.fit(data[feature2],Q000)

    Q00 = lm0_0.predict(preddat000[feature2])
    Q11 = lm0_1.predict(preddat111[feature2])

    lm_00 = linear_model.LinearRegression().fit(data[feature1],Q00)
    lm_11 = linear_model.LinearRegression().fit(data[feature1],Q11)

    phi_gcomp_lm = (np.mean(lm_11.predict(preddat11[feature1])-lm_00.predict(preddat00[feature1])))



    gcomplm_boot = [];   
    msmlm_boot = []; msmlm_b1boot = []; msmlm_b2boot = []; msmlm_b3boot = []
    # gcomp_boot = []; gcomp_b1boot = []; gcomp_b2boot = []
    for b in range(B):
        np.random.seed(233+b)
        sampleid = np.random.choice(range(1, n+1), size=n, replace=True)
        dat_boot = data.iloc[sampleid-1, :]
        bootpreddat11 = dat_boot.copy(); bootpreddat11[trt1]=1;bootpreddat11[trt2]=1
        bootpreddat00 = dat_boot.copy(); bootpreddat00[trt1]=0;bootpreddat00[trt2]=0
        bootpreddat10 = dat_boot.copy(); bootpreddat10[trt1]=1;bootpreddat10[trt2]=0
        bootpreddat01 = dat_boot.copy(); bootpreddat01[trt1]=0;bootpreddat01[trt2]=1

        bootpreddat000 = dat_boot.copy(); bootpreddat000[trt1]=0;bootpreddat000[trt2]=0;bootpreddat000[trt3]=0
        bootpreddat111 = dat_boot.copy(); bootpreddat111[trt1]=1;bootpreddat111[trt2]=1;bootpreddat111[trt3]=1

        pred_boot1 = dat_boot.copy(); pred_boot1[trt2]=1
        pred_boot0 = dat_boot.copy(); pred_boot0[trt2]=0

        ## G-comp with LR
        lm1boot = linear_model.LinearRegression()
        lm1boot.fit(dat_boot[feature3],dat_boot[out3])
        Q111boot = lm1boot.predict(bootpreddat111[feature3])
        Q000boot = lm1boot.predict(bootpreddat000[feature3])

        lm0_1boot = linear_model.LinearRegression()
        lm0_1boot.fit(dat_boot[feature2],Q111boot)
        lm0_0boot = linear_model.LinearRegression()
        lm0_0boot.fit(dat_boot[feature2],Q000boot)

        Q00boot = lm0_0boot.predict(bootpreddat000[feature2])
        Q11boot = lm0_1boot.predict(bootpreddat111[feature2])

        lm_00boot = linear_model.LinearRegression().fit(dat_boot[feature1],Q00boot)
        lm_11boot = linear_model.LinearRegression().fit(dat_boot[feature1],Q11boot)
        gcomplm_boot.append(np.mean(lm_11boot.predict(bootpreddat11[feature1])-lm_00boot.predict(bootpreddat00[feature1])))

        ## MSM with LR
        lgboot0 = linear_model.LogisticRegression().fit(dat_boot[ps0feature],dat_boot[trt1])
        lmpsboot0 = np.clip(lgboot0.predict_proba(dat_boot[ps0feature])[:,1].ravel(),0.01,0.99)
        lgboot1 = linear_model.LogisticRegression().fit(dat_boot[ps1feature],dat_boot[trt2])
        lmpsboot1 = np.clip(lgboot1.predict_proba(dat_boot[ps1feature])[:,1].ravel(),0.01,0.99)
        lgboot2 = linear_model.LogisticRegression().fit(dat_boot[ps2feature],dat_boot[trt3])
        lmpsboot2 = np.clip(lgboot2.predict_proba(dat_boot[ps2feature])[:,1].ravel(),0.01,0.99)

        # lmP = (abcd_encoded[trttp1[0]]/lmps0+(1-abcd_encoded[trttp1[0]])/(1-lmps0))*(abcd_encoded[trttp2[0]]/lmps1+(1-abcd_encoded[trttp2[0]])/(1-lmps1))*(abcd_encoded[trttp3[0]]/lmps2+(1-abcd_encoded[trttp3[0]])/(1-lmps2))
        lmPboot = (dat_boot[trt1].values.ravel()/lmpsboot0+(1-dat_boot[trt1].values.ravel())/(1-lmpsboot0))*(dat_boot[trt2].values.ravel()/lmpsboot1+(1-dat_boot[trt2].values.ravel())/(1-lmpsboot1))*(dat_boot[trt3].values.ravel()/lmpsboot2+(1-dat_boot[trt3].values.ravel())/(1-lmpsboot2))
        msm_lgboot = linear_model.LinearRegression().fit(dat_boot[[trt1,trt2,trt3]],dat_boot[out3],sample_weight=lmPboot)
        msmlm_boot.append(np.sum(msm_lgboot.coef_))
        msmlm_b1boot.append(msm_lgboot.coef_[0][0]); msmlm_b2boot.append(msm_lgboot.coef_[0][1]);msmlm_b3boot.append(msm_lgboot.coef_[0][2])

    gcomplm_se = np.std(gcomplm_boot)
    msmlm_se = np.std(msmlm_boot); msmlmb1_se = np.std(msmlm_b1boot); msmlmb2_se = np.std(msmlm_b2boot); msmlmb3_se = np.std(msmlm_b3boot)

       

    return  phi_dr, beta1, beta2,phi_gcomp_lm, dr_se, drb2_se,drb1_se, gcomplm_se,msmlm_se,msmlmb1_se,msmlmb2_se,drb3_se,beta3,msmlmb3_se

In [ ]:
from functools import partial
start_time = time.time()
#np.random.seed(233)
with multiprocess.Pool(processes=30) as pool:
    #np.random.seed(233)
    result_abcdm = pool.starmap(partial(dr_abcd_boot3,trt1 = trttp1,trt2 = trttp2,trt3 = trttp3,enps0_tr = enps0_tr, 
                                       enps0_ts = enps0_ts,enps1_tr = enps1_tr,enps1_ts = enps1_ts,enps2_tr = enps2_tr,
                                       enps2_ts = enps2_ts,covtp3 = covtp3,ps2feature = ps2feature,
                                       data = abcd_encoded,covbase = covbase,
                                            covtp1 = covtp1,covtp2 = covtp2,ps0feature = ps0feature, 
                                            ps1feature = ps1feature), zip(output1,output2,output3))
    pool.close(); pool.join()

end_time = time.time()
(end_time-start_time)